# SQL Generation with Transformer API

In [ ]:
!pip install torch transformers bitsandbytes accelerate sqlparse

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
torch.cuda.is_available()

True

In [4]:
available_memory = torch.cuda.get_device_properties(0).total_memory

In [5]:
print(available_memory)

15828320256


##Download the Model
Use any model on Colab (or any system with >30GB VRAM on your own machine) to load this in f16. If unavailable, use a GPU with minimum 8GB VRAM to load this in 8bit, or with minimum 5GB of VRAM to load in 4bit.

This step can take around 5 minutes the first time. So please be patient :)

In [6]:
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if available_memory > 15e9:
    # if you have atleast 15GB of GPU memory, run load the model in float16
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
    )
else:
    # else, load in 8 bits – this is a bit slower
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        # torch_dtype=torch.float16,
        load_in_8bit=True,
        device_map="auto",
        use_cache=True,
    )

tokenizer_config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

##Set the Question & Prompt and Tokenize
Feel free to change the schema in the prompt below to your own schema

In [7]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
   name VARCHAR(50), -- Name of the customer
   address VARCHAR(100) -- Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Unit price charged by supplier
);

-- sales.product_id can be joined with products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

##Generate the SQL
This can be excruciatingly slow on a T4 in Colab, and can take 10-20 seconds per query. On faster GPUs, this will take ~1-2 seconds

Ideally, you should use `num_beams`=4 for best results. But because of memory constraints, we will stick to just 1 for now.

In [8]:
import sqlparse

def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [9]:
question = "What was our revenue by product in the New York region last month?"
generated_sql = generate_query(question)

In [10]:
print(generated_sql)


SELECT p.product_id,
       SUM(s.quantity * p.price) AS revenue
FROM sales s
JOIN salespeople sp ON s.salesperson_id = sp.salesperson_id
JOIN products p ON s.product_id = p.product_id
WHERE sp.region = 'New York'
  AND s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
GROUP BY p.product_id
ORDER BY revenue DESC NULLS LAST;


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [11]:
# Prompt: School Database SQL Generator
prompt_school = """
### Task
Generate a SQL query to answer the following question using the given school database schema.

### Instructions
- Only use the fields provided in the schema.
- If the question cannot be answered with the available data, respond with: 'I do not know'.
- Assume standard relationships between tables unless otherwise noted.

### Database Schema

CREATE TABLE students (
  student_id INTEGER PRIMARY KEY, -- Unique ID for each student
  name VARCHAR(50),              -- Full name of the student
  age INTEGER,                   -- Age of the student
  grade_level INTEGER            -- Grade level (e.g., 9, 10, 11, 12)
);

CREATE TABLE subjects (
  subject_id INTEGER PRIMARY KEY, -- Unique ID for each subject
  name VARCHAR(50),              -- Subject name (e.g., Math, History)
  teacher_id INTEGER             -- The teacher assigned to the subject
);

CREATE TABLE teachers (
  teacher_id INTEGER PRIMARY KEY, -- Unique ID for each teacher
  name VARCHAR(50),              -- Full name of the teacher
  department VARCHAR(50)         -- Department the teacher belongs to
);

-- subjects.teacher_id references teachers.teacher_id

### Answer
Below is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""


In [12]:
import sqlparse

def generate_query(question):
    updated_prompt = prompt_school.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [13]:
question1 = "What are the names of all students who are in grade level 12?"
generated_sql1 = generate_query(question1)
print("Generated SQL for Question 1:\n", generated_sql1)


question2 = "Which teachers teach Math or Science subjects?"
generated_sql2 = generate_query(question2)
print("\nGenerated SQL for Question 2:\n", generated_sql2)


question3 = "List the subject name and the department of the teacher who teaches it."
generated_sql3 = generate_query(question3)
print("\nGenerated SQL for Question 3:\n", generated_sql3)


Generated SQL for Question 1:
 
SELECT s.name
FROM students s
WHERE s.grade_level = 12;

Generated SQL for Question 2:
 
SELECT DISTINCT t.name,
                t.department
FROM teachers t
JOIN subjects s ON t.teacher_id = s.teacher_id
WHERE s.name IN ('Math',
                 'Science')

Generated SQL for Question 3:
 
SELECT s.name AS subject_name,
       t.department AS teacher_department
FROM subjects s
JOIN teachers t ON s.teacher_id = t.teacher_id;


**Report Summary**

In this prompt, we tested SQL generation using a Transformer-based model on a school database schema containing students, subjects, and teachers. The prompts were designed to guide the model to create SQL queries based on natural language questions.

- Findings:

  - All three generated SQL queries were accurate and correctly reflected the schema structure.

  - There were no major hallucinations or incorrect assumptions in this case.

  - One small observation was that the model added extra fields (e.g., t.department) even when not explicitly requested — not wrong, but possibly unnecessary depending on the context.

- What I Learned:

  - With a well-structured prompt and clear schema, the model can generate high-quality SQL queries.

  - Explicit instructions (like handling missing data or using aliases) improve the reliability of the output.

  - Prompt design plays a critical role in how precise and relevant the generated SQL is.

Overall, the model performed well, and even small variations in the question style were handled effectively.